based on deep learning with pytorchbook extra lesson

In [1]:
import numpy as np
import torch
torch.set_printoptions(edgeitems=2, threshold=50)

# AUDIO FILES

In [2]:
# getting them

!wget 'https://github.com/deep-learning-with-pytorch/dlwpt-code/raw/master/data/p1ch4/audio-chirp/1-100038-A-14.wav'


--2021-02-02 17:30:27--  https://github.com/deep-learning-with-pytorch/dlwpt-code/raw/master/data/p1ch4/audio-chirp/1-100038-A-14.wav
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/deep-learning-with-pytorch/dlwpt-code/master/data/p1ch4/audio-chirp/1-100038-A-14.wav [following]
--2021-02-02 17:30:28--  https://raw.githubusercontent.com/deep-learning-with-pytorch/dlwpt-code/master/data/p1ch4/audio-chirp/1-100038-A-14.wav
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.64.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.64.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 441044 (431K) [audio/wav]
Saving to: ‘1-100038-A-14.wav’

1-100038-A-14.wav   100%[===================>] 430.71K   724KB/s    in 0.6s    

2021-02-02 17:30:30 (724 KB/s) 

Sound can be seen as fluctuations of pressure of a medium, air for instance, at a certain location in time. There are other representations that we'll get into in a minute, but we can think about this as the raw, time-domain representation. In order for the human ear to appreciate sound, pressure must fluctuate with a frequency between 20 and 20000 oscillations per second (measured in Hertz, Hz). More oscillations per second will lead to a higher perceived pitch.

By recording pressure fluctuations in time using a microphone and converting every pressure level at each time point into a number (e.g. a 16-bit integer), we can now represent sound as a vector of numbers. This is known as Pulse Code Modulation (PCM), where a continuous signal is both sampled in time and quantized in amplitude. If we want to make sure we hear the highest possible pitch in a recording, we'll have to record our samples at slightly more than twice the maximum audible frequency, i.e. just over 40000 times per second. It is not by chance that audio CD's have a sampling frequency of 44100 Hz. This means that a one-hour stereo (i.e. 2 channels) CD track where samples are recorded at 16-bit precision will amount to 2 * 16 * 44100 * 3600 = 5080320000 bit = 605.6 MB if stored without compression.

There are a plethora of audio formats, WAV, AIFF, MP3, AAC being the most popular, where raw audio signals are typically encoded in compressed form by leveraging on both correlation between successive samples in the time series, between the two stereo channels as well as elimination of barely audible frequencies. This can result in dramatic reduction of storage requirements (a one-hour audio file in AAC format takes less than 60 MB). In addition, audio players can decode these formats on the fly on dedicated hardware, consuming a tiny amount of power.

In our data scientist role we may have to feed audio samples to our network and classify them, or generate captions, for instance. In that case, we won't work with compressed data, rather we'll have to find a way to load an audio file in some format and lay it out as an uncompressed time series in a tensor. Let's do that now.

We can download a fair number of environmental sounds at the ESC-50 repository (https://github.com/karoldvl/ESC-50) in the audio directory. Let's get 1-100038-A-14.wav for instance, containing the sound of a bird chirping.

In order to load the sound we resort to SciPy, specifically scipy.io.wavfile.read, which has the nice property to return data as a NumPy array:

In [14]:
import scipy.io.wavfile as wavfile

freq, waveform_arr = wavfile.read('./1-100038-A-14.wav')

In [15]:
freq, waveform_arr

(44100, array([ -388, -3387, -4634, ...,  2289,  1327,    90], dtype=int16))


The read function returns two outputs, namely the sampling frequency and the waveform as a 16-bit integer 1D array. It's a single 1D array, which tells us that it's a mono recording - we'd have two waveforms (two channels) if the sound were stereo.

We can convert the array to a tensor and we're good to go. We might also want to convert the waveform tensor to a float tensor since we're at it

In [16]:
waveform = torch.from_numpy(waveform_arr).float()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729141890/work/torch/csrc/utils/tensor_numpy.cpp:141.)
  """Entry point for launching an IPython kernel.


In [17]:
waveform.shape

torch.Size([220500])

In a typical dataset, we'll have more than one waveform, and possibly over more than one channel. Depending on the kind of network employed for carrying out a task, for instance a sound classification task, we would be required to lay out the tensor in one of two ways.

For architectures based on filtering the 1D signal with cascades of learned filter banks, such as convolutional networks, we would need to lay out the tensor as N x C x L, where N is the number of sounds in a dataset, C the number of channels and L the number of samples in time.

Conversely, for architectures that incorporate the notion of temporal sequences, just as recurrent networks we mentioned for text, data needs to be laid out as L x N x C - sequence length comes first. Intuitively, this is because the latter architectures take one set of C values at a time - the signal is not considered as a whole, but as an individual input changing in time.

Although the most straightforward, this is only one of the ways to represent audio so that it is digestible by a neural network. Anther way is turning the audio signal into a spectrogram.

Instead of representing oscillations explicitly in time, we can characterize what at frequencies those oscillations occur for short time intervals. So, for instance, if we pluck the fifth string of our (hopefully tuned) guitar and we focus on 0.1 seconds of that recording, we will see that the waveform oscillates at 440 cycles per second, plus smaller spurious oscillations at different frequencies that make up the timbre of the sound. If we move on to subsequent 0.1 second intervals, we now see that the frequency content doesn't change, but the intensity does, as the sound of our string fades. If we now decide to pluck another string, we will observe new frequencies fading in time.

We could indeed build a plot having time in the X-axis, frequencies heard at that time in the Y-axis and encode intensity of those frequencies as a value at that X and Y. Or color. Ok, that starts to look like an image, right?

That's correct, spectrograms are a representation of the intensity at each frequency at each point in time. It turns out that one can train convolutional neural networks built for analyzing images (we'll see about those in a couple of chapters) on sound represented as a spectrogram.

Let's see how we can turn the sound we loaded earlier into a spectrogram. To do that, we need to resort to a method for converting a signal in the time domain into its frequency content. This is known as the Fourier transform, and the algorithm that allows us to compute it efficiently is the Fast Fourier Trasform (FFT), which is one of the most widespread algorithms out there. If we do that consecutively for short bursts of sound in time, we can build out spectrogram column by column.

This is the general idea and we won't go into too many details here. Luckily for us SciPy has a function that gets us a shiny spectrogram given an input waveform. We import the signal module from SciPy, then provide the spectrogram function with the waveform and the sampling frequency that we got previously. The return values are all NumPy arrays, namely frequency f_arr (values along the Y axis), time t_arr (values along the X axis) and the actual spectrogra sp_arr as a 2D array. Turning the latter into a PyTorch tensor is trivial:

In [20]:
from scipy import signal

f_arr, t_arr, sp_arr = signal.spectrogram(waveform_arr, freq)
sp_mono = torch.from_numpy(sp_arr)
sp_mono.shape, f_arr, t_arr

(torch.Size([129, 984]),
 array([    0.      ,   172.265625,   344.53125 ,   516.796875,
          689.0625  ,   861.328125,  1033.59375 ,  1205.859375,
         1378.125   ,  1550.390625,  1722.65625 ,  1894.921875,
         2067.1875  ,  2239.453125,  2411.71875 ,  2583.984375,
         2756.25    ,  2928.515625,  3100.78125 ,  3273.046875,
         3445.3125  ,  3617.578125,  3789.84375 ,  3962.109375,
         4134.375   ,  4306.640625,  4478.90625 ,  4651.171875,
         4823.4375  ,  4995.703125,  5167.96875 ,  5340.234375,
         5512.5     ,  5684.765625,  5857.03125 ,  6029.296875,
         6201.5625  ,  6373.828125,  6546.09375 ,  6718.359375,
         6890.625   ,  7062.890625,  7235.15625 ,  7407.421875,
         7579.6875  ,  7751.953125,  7924.21875 ,  8096.484375,
         8268.75    ,  8441.015625,  8613.28125 ,  8785.546875,
         8957.8125  ,  9130.078125,  9302.34375 ,  9474.609375,
         9646.875   ,  9819.140625,  9991.40625 , 10163.671875,
        10335.9


Dimensions are F x T, where F is frequency and T is time.

As we mentioned earlier, stereo sound has two channels, which will lead to a two-channel spectrogram. Suppose we have two spectrograms, one for each channel. We can convert the two channels separately:

In [21]:
sp_left = sp_right = sp_arr
sp_left_t = torch.from_numpy(sp_left)
sp_right_t = torch.from_numpy(sp_right)
sp_left_t.shape, sp_right_t.shape

(torch.Size([129, 984]), torch.Size([129, 984]))

In [22]:
sp_t = torch.stack((sp_left_t, sp_right_t), dim=0)
sp_t.shape

torch.Size([2, 129, 984])

If we want to build a dataset to use as input for a network, we will stack multiple spectrograms representing multiple sounds in a dataset along the first dimension, leading to a N x C x F x T tensor.

Such tensor is indistinguishable from what we would build for a dataset set of images, where F is represents rows and T columns of an image. Indeed, we would tackle a sound classification problem on spectrograms with the exact same networks.

# VIDEO FILES

In [ ]:
torch.set_printoptions(edgeitems=2, threshold=50)

In [23]:
# getting files
!wget 'https://github.com/deep-learning-with-pytorch/dlwpt-code/raw/master/data/p1ch4/video-cockatoo/cockatoo.mp4'

--2021-02-02 17:39:39--  https://github.com/deep-learning-with-pytorch/dlwpt-code/raw/master/data/p1ch4/video-cockatoo/cockatoo.mp4
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/deep-learning-with-pytorch/dlwpt-code/master/data/p1ch4/video-cockatoo/cockatoo.mp4 [following]
--2021-02-02 17:39:40--  https://raw.githubusercontent.com/deep-learning-with-pytorch/dlwpt-code/master/data/p1ch4/video-cockatoo/cockatoo.mp4
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.200.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.200.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 728751 (712K) [application/octet-stream]
Saving to: ‘cockatoo.mp4’

cockatoo.mp4        100%[===================>] 711.67K   939KB/s    in 0.8s    

2021-02-02 17:39:42 (939 

In [27]:
!pip install imageio-ffmpeg -q

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


When it comes to the shape of tensors, video data can be seen as equivalent to volumetric data, with depth replaced by the time dimension. The result is again a 5D tensor with shape N x C x T x H x W.

There are several formats for video, especially geared towards compression by exploiting redundancies in space and time. Luckily for us, imageio reads video data as well. Suppose we'd like to retain 100 consecutive frames in our 512 x 512 RBG video for classifying an action using a convolutional neural network. We first create a reader instance for the video, that will allow us to get information about the video and iterate over the frames in time. Let's see what the meta data for the video looks like:

In [28]:
import imageio

reader = imageio.get_reader('./cockatoo.mp4')
meta =reader.get_meta_data()
meta

{'plugin': 'ffmpeg',
 'nframes': inf,
 'ffmpeg_version': '4.2.2-static https://johnvansickle.com/ffmpeg/ built with gcc 8 (Debian 8.3.0-6)',
 'codec': 'h264',
 'pix_fmt': 'yuv444p',
 'fps': 20.0,
 'source_size': (1280, 720),
 'size': (1280, 720),
 'duration': 14.0}

In [38]:
meta['size']

(1280, 720)

In [44]:
meta['nframes']

inf


In the above, we iterate over individual frames and set each frame in the C x T x H x W video tensor, after transposing the channel. We can then obtain a batch by stacking multiple 4D tensors or pre-allocating a 5D tensor with a known batch size and filling it iteratively, clip by clip, assuming clips are trimmed to a fixed number of frames.

Equating video data to volumetric data is not the only way to represent video for training purposes. This is a valid strategy if we deal with video bursts of fixed length. An alternative strategy is to resort to network architectures capable of processing long sequences and exploiting short and long-term relationships in time, just like for text or audio. // We'll see this kind of architectures when we take on recurrent networks.

This next approach accounts for time along the batch dimension. Hence, we'll build our dataset as a 4D tensor, stacking frame by frame in the batch:

In [56]:
try:
    n_channels = 3
    n_frames = meta['nframes']
    video = torch.empty(n_channels, 1, (1,2))

    print(video.shape)
    
    for i , frame_arr in enumerate(reader):
        frame = torch.from_numpy(frame_arr).float()
        video[:,i] = torch.transpose(frame, 0,2)
    
    time_video = torch.empty(n_frames, n_channels, *meta['size'])

    for i, frame in enumerate(reader):
        frame = torch.from_numpy(frame).float()
        time_video[i] = torch.transpose(frame, 0, 2)

    print(time_video.shape)
except Exception as e:
    print(e)

empty(): argument 'size' must be tuple of ints, but found element of type tuple at pos 3
